# Eval Generation
---
Notebook takes in our eval dataset and generates our various desired eval datasets we can load in.

These will be saved as parquets in the evals folder.

In [1]:
import os
import ast
import json
import random
import pandas as pd

from utils.board import get_piece_name_at_location, convert_board

In [2]:
# =========================================
# Config
# =========================================
PARAMS = {
    'output_dir': "processed_data",
    'max_samples': 5,
    'task': 'eval',      # {'rejsampling', 'eval', 'verl', 'verl_eval'}
    'board_notation': 'uniform_visual',     # {fen, spaced_fen, visual, uniform_visual}
    'use_fixed_ids': False,
    'final_samples_fixed_ids': 10,
    'legalmoves_minmoves': 3,       # If you set = 2, way too many samples are for pawns (which are very easy anyway)
    'worstmove_movethresh': 0.2,
    'worstmove_providedmoves': 5,
    'bestmove_movethresh': 0.2,
    'bestmove_providedmoves': 5,
    'predictmove_minpossiblemoves': 3
}
FILE_MAPPING = {
    "eval": './chess_data/deepmind62k_evals_1k.csv',
    "verl_eval": './chess_data/deepmind62k_evals_1k.csv',
    "verl": './chess_data/deepmind62k_train_50k.csv',
    "rejsampling": './chess_data/deepmind62k_train_50k.csv',
}
SAVED_FILES = []

# =========================================
# Import / process our data
# =========================================
input_filename = FILE_MAPPING[PARAMS['task']]
df = pd.read_csv(input_filename)
df['Move'] = df['Move'].apply(ast.literal_eval)
df['Win Probability'] = df['Win Probability'].apply(ast.literal_eval)

fixed_ids = None
if PARAMS['use_fixed_ids']:
    fixed_ids = random.sample(
        df["FEN ID"].unique().tolist(),
        PARAMS['max_samples']
    )
    pool = df[df["FEN ID"].isin(fixed_ids)]

# Generate Data   
---

### Legal Moves   

In [3]:
# Start by shuffling if not using fixed ids
if not PARAMS['use_fixed_ids']:
    pool = df.sample(frac=1, ignore_index=True)

# Now generate our samples
outputs = []
for index, row in pool.iterrows():
    if not PARAMS['use_fixed_ids'] and len(outputs) >= PARAMS['max_samples']:
        break
    
    board    = row["FEN"]
    moveset  = row["Move"]
    winprobs = row["Win Probability"]
    fen_id   = row["FEN ID"]

    # Get the counts of the various pieces in the moveset
    piece_counts = {}
    for move in moveset:
        piece_pos = move[:2]
        piece_counts[piece_pos] = piece_counts.get(piece_pos, 0) + 1
    valid_pieces = [k for k, v in piece_counts.items() if v >= PARAMS['legalmoves_minmoves']]
    if not valid_pieces:
        continue  # Skip if no valid pieces w/ enough moves

    # Sample a piece from the valid pieces
    piece = random.choice(valid_pieces)
    piece_name = get_piece_name_at_location(board, piece)
    if piece_name is None:
        print(f"Piece not found at {piece} in FEN: {board}")
        continue
    
    user = f"""Below is a board in a game you're currently playing.

{convert_board(board, PARAMS['board_notation'])}

You must provide a list of all legal moves for the {piece_name} at {piece}.

You may want to think out loud to help finalize your answer. However, you must provide your answer within answer tags (e.g., <answer> list_of_moves </answer>).

The moves must be provided as a list, in UCI notation, and within answer tags in order to be accepted."""
    
    chat_history = [
        ['system', "chess_task_sysprompt.txt"], 
        ['user', user], 
        ['assistant', ""], 
    ]
    
    legal_moves = [move for move in moveset if move.startswith(piece)]
    info = {
        'board_id': fen_id,
        'board': board,
        'answer': legal_moves,
        'task_data': (piece_name, piece)
    }
    
    outputs.append({
        'chat': chat_history,
        'info': info
    })

# Export as jsonl
jsonl_path = f"./{PARAMS['output_dir']}/legalmoves_{PARAMS['task']}_{len(outputs)}.jsonl"
with open(jsonl_path, 'w', encoding='utf-8') as f:
    for obj in outputs:
        f.write(json.dumps(obj) + '\n')
print(f"Saved JSONL to {jsonl_path}")
SAVED_FILES.append(jsonl_path)

Saved JSONL to ./processed_data/legalmoves_eval_5.jsonl


### Worst Move

In [4]:
# Start by shuffling if not using fixed ids
if not PARAMS['use_fixed_ids']:
    pool = df.sample(frac=1, ignore_index=True)

# Iterate through and get samples
outputs = []
for index, row in pool.iterrows():
    if not PARAMS['use_fixed_ids'] and len(outputs) >= PARAMS['max_samples']:
        break

    board = row['FEN']
    moveset = row['Move']
    win_probs = row['Win Probability']
    id = row['FEN ID']

    move_prob_pairs = list(zip(moveset, win_probs))
    worst_move, worst_move_win_prob = min(move_prob_pairs, key=lambda x: x[1])

    filtered_moves = [
        move for move, prob in move_prob_pairs
        if prob > PARAMS['worstmove_movethresh'] + worst_move_win_prob
    ]

    if len(filtered_moves) < PARAMS['worstmove_providedmoves'] - 1:
        continue

    sampled_moves = random.sample(filtered_moves, PARAMS['worstmove_providedmoves'] - 1)
    sampled_moves.append(worst_move)
    random.shuffle(sampled_moves)

    user_prompt = f"""Below is a board in a game you're currently playing.

{convert_board(board, PARAMS['board_notation'])}
    
You must choose the worst move from the following moves: {sampled_moves}. 

You may want to think out loud to help finalize your answer. However, you must provide your answer within answer tags (e.g., <answer> worst_move </answer>).

The move must be provided in UCI notation and within answer tags in order to be accepted."""

    chat_history = [
        ['system', "chess_task_sysprompt.txt"],
        ['user', user_prompt],
        ['assistant', ""], 
    ]
    info = {
        'board_id': id,
        'board': board,
        'answer': {'answer': worst_move, 'candidates': sampled_moves}
    }

    outputs.append({
        'chat': chat_history,
        'info': info
    })

# Export as jsonl
jsonl_path = f"./{PARAMS['output_dir']}/worstmove_{PARAMS['task']}_{len(outputs)}.jsonl"
with open(jsonl_path, 'w', encoding='utf-8') as f:
    for obj in outputs:
        f.write(json.dumps(obj) + '\n')
print(f"Saved JSONL to {jsonl_path}")
SAVED_FILES.append(jsonl_path)

Saved JSONL to ./processed_data/worstmove_eval_5.jsonl


### Best Move   

In [5]:
# Start by shuffling if not using fixed ids
if not PARAMS['use_fixed_ids']:
    pool = df.sample(frac=1, ignore_index=True)

# Iterate through and get samples
outputs = []
for index, row in pool.iterrows():
    if not PARAMS['use_fixed_ids'] and len(outputs) >= PARAMS['max_samples']:
        break

    board = row['FEN']
    moveset = row['Move']
    win_probs = row['Win Probability']
    id = row['FEN ID']

    move_prob_pairs = list(zip(moveset, win_probs))

    # Find the best move (highest win probability)
    best_move, best_move_win_prob = max(move_prob_pairs, key=lambda x: x[1])

    # Filter out moves with win probability < best_move_win_prob - best_move_thresh
    filtered_moves = [
        move for move, prob in move_prob_pairs
        if prob < best_move_win_prob - PARAMS['bestmove_movethresh']
    ]

    if len(filtered_moves) < PARAMS['bestmove_providedmoves'] - 1:
        continue

    sampled_moves = random.sample(filtered_moves, PARAMS['bestmove_providedmoves'] - 1)
    sampled_moves.append(best_move)
    random.shuffle(sampled_moves)

    user_prompt = f"""Below is a board in a game you're currently playing.

{convert_board(board, PARAMS['board_notation'])}
    
You must choose the best move from the following moves: {sampled_moves}. 

You may want to think out loud to help finalize your answer. However, you must provide your answer within answer tags (e.g., <answer> best_move </answer>).

The move must be provided in UCI notation and within answer tags in order to be accepted."""

    chat_history = [
        ['system', "chess_task_sysprompt.txt"],
        ['user', user_prompt],
        ['assistant', ""], 
    ]
    info = {
        'board_id': id,
        'board': board,
        'answer': {'answer': best_move, 'candidates': sampled_moves}
    }

    outputs.append({
        'chat': chat_history,
        'info': info
    })

# Export as jsonl
jsonl_path = f"./{PARAMS['output_dir']}/bestmove_{PARAMS['task']}_{len(outputs)}.jsonl"
with open(jsonl_path, 'w', encoding='utf-8') as f:
    for obj in outputs:
        f.write(json.dumps(obj) + '\n')
print(f"Saved JSONL to {jsonl_path}")
SAVED_FILES.append(jsonl_path)

Saved JSONL to ./processed_data/bestmove_eval_5.jsonl


### Predict Move   

In [6]:
# Start by shuffling if not using fixed ids
if not PARAMS['use_fixed_ids']:
    pool = df.sample(frac=1, ignore_index=True)

# Iterate through and get samples
outputs = []
for index, row in pool.iterrows():
    if not PARAMS['use_fixed_ids'] and len(outputs) >= PARAMS['max_samples']:
        break

    board = row['FEN']
    moveset = row['Move']
    win_probs = row['Win Probability']
    id = row['FEN ID']

    if len(moveset) < PARAMS['predictmove_minpossiblemoves']:
        continue

    move_prob_dict = dict(zip(moveset, win_probs))

    user_prompt = f"""Below is a chess board from your current game.

{convert_board(board, PARAMS['board_notation'])}

You must select the best move from this position and return it within answer tags. Your answer must be formatted as <answer> my_move </answer>, where my_move is a legal move in UCI notation.

Think step by step if necessary, but do not omit the answer tags or UCI format. Only answers in the correct format will be accepted."""

    chat_history = [
        ['system', "chess_task_sysprompt.txt"],
        ['user', user_prompt],
        ['assistant', ""], 
    ]

    info = {
        'board_id': id,
        'board': board,
        'answer': move_prob_dict
    }

    outputs.append({
        'chat': chat_history,
        'info': info
    })

# Export as jsonl
jsonl_path = f"./{PARAMS['output_dir']}/predictmove_{PARAMS['task']}_{len(outputs)}.jsonl"
with open(jsonl_path, 'w', encoding='utf-8') as f:
    for obj in outputs:
        f.write(json.dumps(obj) + '\n')
print(f"Saved JSONL to {jsonl_path}")
SAVED_FILES.append(jsonl_path)

Saved JSONL to ./processed_data/predictmove_eval_5.jsonl
